In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data = pd.read_csv("../../../data/aapl_1d_train.csv")
rsi_data = ta.momentum.RSIIndicator(data.Close, window=14)
data["RSI"] = rsi_data.rsi()
data = data.dropna()

In [3]:
def def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral):
    cash= 1_000_000
    com = 0.125/100
    history = []
    portfolio_value = []
    active_operations = []
    total_profit_loss = 0

    for i, row in data.iterrows():
        # Close active operation
        active_op_temp = []
        for operation in active_operations:
            if operation["stop_loss"] > row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"profit_loss": profit_loss})
            elif operation["take_profit"] < row.Close:
                cash += (row.Close * operation["n_shares"]) * (1 - com)
                # Calcular la ganancia o pérdida y actualizar el total
                profit_loss = (row.Close * operation["n_shares"]) * (1 - com) - (operation["bought"] * operation["n_shares"])
                total_profit_loss += profit_loss
                history.append({"profit_loss": profit_loss})
            else:
                active_op_temp.append(operation)
        active_operations = active_op_temp
            
        # ¿Tenemos suficiente efectivo?
        if cash < (row.Close * n_shares * (1 + com)):
            asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
            portfolio_value.append(cash + asset_vals)
            continue
        
        # Analizar la señal larga
        if row.RSI <= Umbral:  # If ceil(Op)
            active_operations.append({
                "bought": row.Close,
                "n_shares": n_shares,
                "type": "long",
                "stop_loss": row.Close * stop_loss_close,
                "take_profit": row.Close * take_profit_close
            })
            cash -= row.Adjusted_close * n_shares * (1 + com)
        
        asset_vals = sum([operation["n_shares"] * row.Close for operation in active_operations])
        portfolio_value.append(cash + asset_vals)
    
    return total_profit_loss

In [4]:
import optuna

def objective(trial):
    # Define los rangos de búsqueda para los parámetros
    n_shares = trial.suggest_int('n_shares', 1, 50)
    stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
    take_profit_close = trial.suggest_uniform('take_profit_close', 1, 1.5)
    Umbral = trial.suggest_uniform('Umbral', -10, 10)

    # Evalúa la función de trading con los parámetros sugeridos
    profit_loss = def_trading_strategy(data, n_shares, stop_loss_close, take_profit_close, Umbral)
    
    return profit_loss  # Devuelve la métrica que se debe minimizar/maximizar

# Crea un estudio Optuna
study = optuna.create_study(direction='maximize')  # 'maximize' si se quiere maximizar la métrica

# Ejecuta la optimización
study.optimize(objective, n_trials=20)  # Puedes ajustar n_trials según tus recursos y tiempo disponible

# Obtiene los mejores parámetros encontrados
best_params = study.best_params
best_profit_loss = study.best_value

print("Mejores parámetros encontrados:")
print(best_params)
print("Rendimiento (Profit/Loss):", best_profit_loss)


[I 2024-02-25 21:43:12,852] A new study created in memory with name: no-name-0d49b2dc-9a85-4585-beec-683ddc8c5105
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_17318/1268521632.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  stop_loss_close = trial.suggest_uniform('stop_loss_close', 0.80, 1)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_17318/1268521632.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  take_profit_close = trial.suggest_uniform('take_profit_close', 1, 1.5)
/var/folders/x7/frl5my251vx198wg4w_ndzxh0000gn/T/ipykernel_17318/1268521632.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.co

Mejores parámetros encontrados:
{'n_shares': 11, 'stop_loss_close': 0.9139912799835698, 'take_profit_close': 1.3858058448500528, 'Umbral': 9.706663497509684}
Rendimiento (Profit/Loss): 6394.193437500001


In [5]:
# Condiciones para comprar y vender
condicion_compra = data["RSI"] <= -1.5
condicion_venta = data["RSI"] > 1.1

# Aplicando las condiciones para obtener señales de compra y venta
data['Senal_rsi'] = ''
data.loc[condicion_compra, 'Senal_rsi'] = 'Compra'
data.loc[condicion_venta, 'Senal_RSI'] = 'Venta'
data.to_excel('../../../data/1d/RSI_BOS.xlsx', index=False)

In [6]:
data['Senal_RSI'].value_counts()

Senal_RSI
Venta    5271
Name: count, dtype: int64